### Optimize parameters of 2D Lacey model using several optimizers
- [ ] define a optimization function (receiving normalized parameters)
- [ ] define a parameter normalization function
- [ ] define a optimize function that receives parameter ranges
- [ ] test several optimizers

In [ ]:
import orprofile
from orprofile.api.depth import _prepare_ds, _prepare_B_grid, _compute_depth
import geopandas as gpd
import os
import xugrid as xu
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

### Load data into a Mesh object

In [ ]:
fn_splines = "../examples/data/splines.geojson"
fn_points = "../examples/data/bamboi_survey.geojson"
splines = gpd.read_file(fn_splines)

mesh = orprofile.api.mesh.Mesh(splines, n=20, m=20)
mesh.plot()
gdf = gpd.read_file(fn_points)

# prepare a meshed set
ds = orprofile.api.regrid_samples(mesh, gdf)
ds["samples"].ugrid.plot()
samples = ds["samples"].values

In [ ]:
da = ds["samples"].ugrid.laplace_interpolate()
da.ugrid.plot()

In [ ]:
alpha = 30  # amplitude of meander around centerline [m]
L = 300  # m
A = 500  # conveyance m2
c = 0.5  # width to grid width ratio [-]
kappa = 0.5*np.pi  # phase [rad]

alpha = 63.49418969047191
L = 688.457688989413
A = 1222.7566649899356
c = 1.5903139437728133
kappa = -1.1495183526701667
ds_depth = orprofile.api.depth_2d(mesh, alpha, L, kappa, c, A)

print(ds_depth["depth"].values.mean())
residuals = ds_depth["depth"].values - samples
residuals = residuals[np.isfinite(residuals)]

residuals.std()


In [ ]:
import copy
def error_depth(x, mesh, samples, ds, B_grid, x_mult, x_off):
    """
    """
    alpha, L, kappa, c, A = np.array(x) * x_mult + x_off
    # print(alpha, L, kappa, c, A)
    depth = _compute_depth(copy.deepcopy(ds), B_grid, alpha, L, kappa, c, A)["depth"].values
    # depth = orprofile.api.depth_2d(mesh, alpha, L, kappa, c, A)["depth"].values
    # print(depth.mean())
    residuals = depth - samples
    residuals = residuals[np.isfinite(residuals)]
    mse = np.mean(residuals**2)
    print(np.sqrt(mse))
    return mse

alpha = (10, 100)
L = (15, 200)
A = (20, 2000)
c = (0.5, 2)
kappa = (-0.5*np.pi, 0.5*np.pi)

x_range = [alpha, L, kappa, c, A]
x_off = np.array([_x[0] for _x in x_range])
x_mult = np.array([_x[1] - _x[0] for _x in x_range])
x_lim = [(0, 1) for _ in x_range]

ds = _prepare_ds(mesh)
B_grid = _prepare_B_grid(ds)


opt = optimize.differential_evolution(
    error_depth,
    bounds=x_lim,
    args=(mesh, samples - 116.5, ds, B_grid, x_mult, x_off),
    atol=1,  # 1 meter
    maxiter=100,
)


In [ ]:
opt.x * x_mult + x_off

In [ ]:
opt.x
alpha, L, kappa, c, A = np.array(opt.x) * x_mult + x_off
ds_depth = orprofile.api.depth_2d(mesh, alpha, L, kappa, c, A)

print(ds_depth["depth"].values.mean())
residuals = samples - ds_depth["depth"]
residuals -= residuals.mean()
residuals2 = residuals.ugrid.laplace_interpolate()
f, axs = plt.subplots(ncols=3, figsize=(13, 5))

ds_depth["depth"].ugrid.plot(ax=axs[0])
residuals.ugrid.plot(ax=axs[1])
residuals2.ugrid.plot(ax=axs[2])



In [ ]:
da_depth = ds_depth["depth"] + residuals2
da_depth.ugrid.plot(vmax=0)

In [ ]:
da = ds["samples"]
da -= da.mean()
da2 = da.ugrid.laplace_interpolate()
da2.ugrid.plot()

### Reduce the average depth per grid cell to a single value and map these onto the grid

In [ ]:
(da2 - da_depth).ugrid.plot()

In [ ]:
points = da_index.ugrid.sel_points(x=mesh.points.geometry.x, y=mesh.points.geometry.y)
ax = plt.axes()
# da_index.ugrid.plot(ax=ax)
ax.scatter(points.mesh2d_x, points.mesh2d_y, c=points)

In [ ]:
# add points to gdf (nans where no data)
gdf = mesh.points
face_index = np.ones(len(gdf), dtype=np.int64)*-1
face_index[points.mesh2d_index] = points.values
gdf["face_index"] = face_index
# gdf.plot(column="face_index")
gdf

In [ ]:
# reduce per index
depth_mean = gdf[gdf["face_index"] >= 0][["Depth", "face_index"]].groupby("face_index").mean()
# add data to new xu.UgridDataArray

### organizing rows and columns into a XUGridDataSet

In [ ]:
depth_mean.index
depth_grid = np.zeros(da_index.shape)*np.nan
depth_grid[depth_mean.index] = depth_mean["Depth"]
depth_grid

In [ ]:
ds = mesh._get_empty_ds()
ds["samples"] = ("mesh2d_nFaces", depth_grid)
ds
# ds["samples"].ugrid.plot()

f = plt.figure(figsize=(16, 9))
ax = mesh.plot()
ds = ds.ugrid.to_crs(ax.projection)
ds["samples"].ugrid.plot(ax=ax, alpha=0.8)
plt.show()

In [ ]:
ax = plt.axes()
ds["samples"][ds["rows"]==8].plot(ax=ax, marker=".")
ds["samples"][ds["rows"]==9].plot(ax=ax, marker=".")
ds["samples"][ds["rows"]==12].plot(ax=ax, marker=".")
ds["samples"][ds["rows"]==14].plot(ax=ax, marker=".")
ds["samples"][ds["rows"]==15].plot(ax=ax, marker=".")
ds["samples"][ds["rows"]==20].plot(ax=ax, marker=".", label=19)
ax.legend()
ax.axis("equal")

In [ ]:
da.ugrid.sel_points(x=606700, y=901475)

In [ ]:
mesh.mesh2d.to_dataset().mesh2d_nNodes